<a href="https://colab.research.google.com/github/Emilevillette/Emilevillette/blob/main/SIMT_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LINFO2241 : Practical Session 8

## Exercice 2 : SIMT

### Section 1 : Converting a simple program into CUDA

First, install dependencies

In [1]:
!pip install git+http://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning http://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-haub_x2t
  Running command git clone --filter=blob:none --quiet http://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-haub_x2t
  Resolved http://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=0066a6147b868ad0c023e0780bb9f0359d28f76eedd917afb5d10fa3d16f27c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-xc1t8liz/wheels/c8/7e/06/e62d8d9c02dd325871935b2afda42a4784814746d313cf308d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


Now, remember this program you used in the previous exercise :

```c
#include <stdio.h>
#include <stdlib.h>


long N = 512;

// Naive implementation
void VecAdd(float* A, float* B, float* C)
{
    for (int i = 0; i < N; i++)
        C[i] = A[i] + B[i];
}

// Host code
int main()
{

    size_t size = N * sizeof(float);

    // Allocate input vectors h_A and h_B in host memory
    float* A = (float*)malloc(size);
    float* B = (float*)malloc(size);
    float* C = (float*)malloc(size);

    // Initialize input vectors
    for (int i = 0; i < N; i ++) {
        A[i] = (float)i;
        B[i] = (float)i;
        //C[i] = 0; will be overwritten, don't care
    }

    // Start computation
    printf("Launching computation...\n");
    unsigned long long start = rdtscl();
    VecAdd(A, B, C, N);
    printf("Finished in %llu cycles !\n", rdtscl() - start);

    printf("First floats of C : %f %f %f ...\n", C[0], C[1] , C[2]);

    // Free host memory
    free(A);
    free(B);
    free(C);

    printf("Exiting...");
}
```

Your job is to convert it step by step into a GPU compatible program. Follow each steps carefully. You can use the last box to write your findings step-by-step.

#### Step 1 : Allocating memory on the device

The first step in translating this function into cuda is to allocated memory on the device (i.e. The GPU). There are special functions for that.

[Hint](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356)

```c
// Allocate input vectors h_A and h_B in host memory
float* cpu_A = (float*)malloc(size);
float* cpu_B = (float*)malloc(size);
float* cpu_C = (float*)malloc(size);

// Initialize input vectors
for (int i = 0; i < N; i ++) {
    cpu_A[i] = (float)i;
    cpu_B[i] = (float)i;
    //C[i] = 0; will be overwritten, don't care
}

// Now, allocate 3 arrays of float, equivalent of cpu_A,cpu_B and cpu_C, but on the GPU
float* gpu_A;
float* gpu_B;
float* gpu_C;
...

// Start computation
printf("Launching computation...\n");
```

#### Step 2 : Copying memory from the CPU to the device


Now that you allocated your memory on the GPU, you can copy arrays A and B into GPU memory ! There is a simple function to perform this copy, you just have to call it twice.

[Hint](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8)

```c
// Allocate input vectors h_A and h_B in host memory
float* cpu_A = (float*)malloc(size);
float* cpu_B = (float*)malloc(size);
float* cpu_C = (float*)malloc(size);

// Initialize input vectors
for (int i = 0; i < N; i ++) {
    cpu_A[i] = (float)i;
    cpu_B[i] = (float)i;
    //C[i] = 0; will be overwritten, don't care
}

// Code from previous Exercice : allocate 3 arrays of float
float* gpu_A;
float* gpu_B;
float* gpu_C;
...

// Exercice 2 : Copy cpu_A and cpu_B into gpu_A and gpu_B
...

// Start computation
printf("Launching computation...\n");
```

## Interlude: Manageing threads in GPUs

Before going further, it is important to understand how threads are handled in GPUs.

A CUDA thread takes care of a single element of data. As seen in the course, the idea of SIMT is to execute many many threads in parallel. Threfore threads are organized into blocks of 32 threads. And the whole computation to be done, is an ensemble of "blocks" to be executed.

![GPU blocks](https://developer-blogs.nvidia.com/wp-content/uploads/2020/06/kernel-execution-on-gpu-1-625x438.png)

A thread block will be executed by a CUDA Streaming Multiprocessor (SM). It is like the SIMT design seen in the lecture : a single instruction pointer is shared among all threads, advancing on all their independent data element.


Let's say you have an array of 2048 elements. You will have to divide it in blocks of a multiple of 32 threads (let's say 32 for now). You will therefore have 64 blocks of 32 threads. The details will be seen in the course, but the NVIDIA GigaThread will assign each block to the CUDA SM. It is a hardware piece, this is not done in software by you or the driver.

In our case, a thread does not loop. With CUDA, you just launch many thread that works individually in a scalar way on a single element. But which scalar? In your array of 2048 elements, how to know which thread should access `array[0]` or `array[258]`, if all threads execute the same code ?

By using the thread index !

CUDA exposes a few global objects. blockDim, blockIdx and threadIdx. Those object will actually have different values for each different threads.
In this example we will only use a single dimension, so we will limit ourselves to ".x" of each element.

 * blockDim.x <-- the wideness of the block (always 32 as we use 32 threads per blocks)
 * blockIdx.x <-- the index of the block (from 0 to 63, as we have 64 blocks in total)
 * threadIdx.x <-- the index of the thread (from 0 to 31, as there is 32 threads per blocks)

In practice, one can change the size of the blocks. You will try that later.

So, how to compute, in an array of 2048 a unique index for all threads, so that at the end, by launching all 64 blocks of 32 threads, all 2048 elements have been computed ?

`int i = ...`

#### Step 3 : Adapting the function


The objective of this function is to run solely on the GPU, all you have is the 3 vectors you initialized and the value `N`, the maximal size of the array. As this function starts with `__global__`, it will run on the GPU. Now, you must take advantage of GPU architecture.

Following the discussion in the last block of text, you can compute on which index of A, B and C this thread should operate.

One last note : you must verify that the computed index is not bigger than N ! Indeed, if N is not a multiple of 32, then the last threads in the block would work out of bound !


```c
/**
* @param gpu_A : Array A allocated on the GPU
* @param gpu_B : Array B allocated on the GPU
* @param gpu_C : Array C allocated on the GPU
* @param N : Length of each array
*/
__global__ void VecAdd(float* gpu_A, float* gpu_B, float* gpu_C, long N)
{
  // Your code here
}
```

#### Step 4 : Changing the function call


Functions that runs on GPU aren't called normally. We give you this part of the code, but you'll have to tweak parameters such as `threadsPerBlock` and `blocksPerGrid`.


```c
    printf("Launching kernel...\n");
    // Invoke GPU function
    unsigned long long start = rdtscl();
    int threadsPerBlock = 128; //Always a multiple of 32 !
    int blocksPerGrid =
            (N + threadsPerBlock - 1) / threadsPerBlock;
    VecAdd<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, N);

    printf("Finished in %llu cycles !\n", rdtscl() - start);
```

#### Step 5 : Copying result from memory to the CPU


Now that you called you GPU function and it successfully stored the result of its computations within `gpu_C`, you simply have to copy its content into CPU memory (The opposite of what you did at step 2)


```c
    printf("Launching kernel...\n");
    // Invoke GPU function
    unsigned long long start = rdtscl();
    int threadsPerBlock = 128;
    int blocksPerGrid =
            (N + threadsPerBlock - 1) / threadsPerBlock;
    VecAdd<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, N);

    // Copying gpu_C into cpu_C
    ...

    
    printf("Finished in %llu cycles !\n", rdtscl() - start);
```

#### Step 6 : Freeing memory

In this final step, as every respectful developer, you must free your memory.

```c
    printf("First floats of C : %f %f %f ...\n", h_C[0], h_C[1] , h_C[2]);
    
    // Free device memory
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);

    // Free host memory
    free(cpu_A);
    free(cpu_B);
    free(cpu_C);

    printf("Exiting...");
```

#### Final code

This cell will contain your final code, we recommand that you progressively complete it by following each step described above :

In [31]:
%%writefile cuda.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdint.h>

unsigned long long rdtscl(void)
{
    unsigned int lo, hi;
    __asm__ __volatile__ ("rdtsc" : "=a"(lo), "=d"(hi));
    return ( (unsigned long long)lo)|( ((unsigned long long)hi)<<32 );
}

// Device code
__global__ void VecAdd(float* gpu_A, float* gpu_B, float* gpu_C, int N)
{
    // CUDA version of the function
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {gpu_C[i] = gpu_A[i] + gpu_B[i];}
}

// Host code
int main()
{
    uint64_t N = 512;
    size_t size = N * sizeof(float);

    // Allocate input vectors h_A and h_B in host memory
    float* cpu_A = (float*)malloc(size);
    float* cpu_B = (float*)malloc(size);
    float* cpu_C = (float*)malloc(size);

    // Initialize input vectors
    for (int i = 0; i < N; i ++) {
        cpu_A[i] = (float)i;
        cpu_B[i] = (float)i;
        //h_C[i] = 0; will be overwritten, don't care
    }

    // Allocate vectors in device memory
    float* gpu_A;
    float* gpu_B;
    float* gpu_C;

    cudaMalloc( &gpu_A, size );
    cudaMalloc( &gpu_B, size );
    cudaMalloc( &gpu_C, size );

    // Copy vectors from host memory to device memory
    cudaMemcpy( gpu_A,cpu_A, size, cudaMemcpyDefault);
    cudaMemcpy( gpu_B,cpu_B, size, cudaMemcpyDefault);
    cudaMemcpy( gpu_C, cpu_C, size, cudaMemcpyDefault);


    printf("Launching kernel...\n");
    // Invoke GPU function
    unsigned long long start = rdtscl();
    int threadsPerBlock = 128;
    int blocksPerGrid =
            (N + threadsPerBlock - 1) / threadsPerBlock;
    VecAdd<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, N);

    // Copy result from GPU to CPU
    cudaMemcpy(cpu_C, gpu_C, size, cudaMemcpyDefault);


    printf("Finished in %llu cycles !\n", rdtscl() - start);

    printf("First floats of C : %f %f %f ...\n", cpu_C[0], cpu_C[1] , cpu_C[2]);

    // Free device memory
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);

    // Free host memory
    free(cpu_A);
    free(cpu_B);
    free(cpu_C);

    printf("Exiting...");
}

Overwriting cuda.cu


In [32]:
!nvcc -o cuda cuda.cu

In [33]:
!./cuda || echo "Crashed with code : $?"

Launching kernel...
Finished in 84188 cycles !
First floats of C : 0.000000 2.000000 4.000000 ...
Exiting...

### Section 2 : Performance

Compare the CUDA implementation and the normal one. Can you see a difference? Can you tweak CUDA parameters to change performance?

In [34]:
%%writefile naive.c
#include <stdio.h>
#include <stdlib.h>

unsigned long long rdtscl(void)
{
    unsigned int lo, hi;
    __asm__ __volatile__ ("rdtsc" : "=a"(lo), "=d"(hi));
    return ( (unsigned long long)lo)|( ((unsigned long long)hi)<<32 );
}

// Naive implementation
void VecAdd(float* A, float* B, float* C, long N)
{
    for (int i = 0; i < N; i++)
        C[i] = A[i] + B[i];
}

// Host code
int main()
{

    long N = 512;
    size_t size = N * sizeof(float);

    // Allocate input vectors h_A and h_B in host memory
    float* A = (float*)malloc(size);
    float* B = (float*)malloc(size);
    float* C = (float*)malloc(size);

    // Initialize input vectors
    for (int i = 0; i < N; i ++) {
        A[i] = (float)i;
        B[i] = (float)i;
        //C[i] = 0; will be overwritten, don't care
    }

    // Start computation

    printf("Launching computation...\n");
    unsigned long long start = rdtscl();
    VecAdd(A, B, C, N);

    printf("Finished in %llu cycles !\n", rdtscl() - start);

    printf("First floats of C : %f %f %f ...\n", C[0], C[1] , C[2]);

    // Free host memory
    free(A);
    free(B);
    free(C);

    printf("Exiting...");
}

Writing naive.c


In [35]:
!gcc -march=native naive.c -o naive

In [36]:
!./naive
!./cuda

Launching computation...
Finished in 4882 cycles !
First floats of C : 0.000000 2.000000 4.000000 ...
Exiting...Launching kernel...
Finished in 82122 cycles !
First floats of C : 0.000000 2.000000 4.000000 ...
Exiting...

What do you conclude on the performance of your CUDA version? Why? Also try to change the threadsPerBlock to 1, 4 or 32. What do you observe? Why?

# Going further : Matrix multiplication

In the previous exercise we worked with a one-dimensional vector. Let's now consider square matrix, a vector with two dimension. CUDA has support for blocks of 2 and 3 dimensions.

For a matrix multiplication, composed of 3 loops, the idea is to have each thread do the internal loop using the naive matrix multiplication. Indeed, the optimized version we saw requires complex synchronization to add results from different threads. This is beyond this course.

Consider a matrix with dimension `N=128`. We are limited by the size of blocks. It would be easy to simply launch a block of size `128*128`, but that is not possible. We will use blocks of size `32*32`.

We give you the code to launch the Kernel. You must compute the number of times a block must be launched horizontally (grid_rows) and vertically (grid_cols).

```c
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(threadsPerBlock, threadsPerBlock);
    MatMult<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C, N);
```

Then, use blockIdx.x, blockDim.x and threadIdx.x to compute the row inside the MatMult function, and blockIdx.y, blockDim.y and threadIdx.y to compute the column index.

The goal of MatMult is to set `gpu_C[row * N + col]` to the sum of the product of the row of A and the col of B. Be sure to check for bounds !